<h2> This code uses Stable baselines3 to train the model instead of mjrl </h2>

Import modules

In [1]:
import myosuite
import gym
import skvideo.io
import numpy as np
import os

MyoSuite:> Registering Myo Envs


Display settings

In [2]:
from IPython.display import HTML
from base64 import b64encode

def show_video(video_path, video_width = 400):
  video_file = open(video_path, "r+b").read()
 
  video_url = f"data:video/mp4;base64,{b64encode(video_file).decode()}"
  return HTML(f"""<video autoplay width={video_width} controls><source src="{video_url}"></video>""")

<h2> Make environment </h2>
The environment should exist in \envs\myo\ of the myosuite module (either in myobase or myochallenge)

In [3]:
env = gym.make('CenterReachOut1-v0')
env.set_perturbation_force(0)
env.reset()
env.set_perturbation_force(0)

    MyoSuite: A contact-rich simulation suite for musculoskeletal motor control
        Vittorio Caggiano, Huawei Wang, Guillaume Durandau, Massimo Sartori, Vikash Kumar
        L4DC-2019 | https://sites.google.com/view/myosuite
    


In [4]:
from stable_baselines3.common.logger import configure

tmp_path = "/tmp"
# set up logger
new_logger = configure(tmp_path, ["stdout", "csv", "tensorboard"])

pygame 2.5.2 (SDL 2.28.3, Python 3.8.18)
Hello from the pygame community. https://www.pygame.org/contribute.html
Logging to /tmp


In [5]:
from stable_baselines3 import PPO
import os
from stable_baselines3.common.noise import NormalActionNoise

# Setup logging directory
logdir = "logs"
models_dir = "models/SACarmreach" 
#n_actions = env.action_space.shape[-1]
#action_noise = NormalActionNoise(mean=np.zeros(n_actions), sigma=0.1 * np.ones(n_actions))
model = PPO("MlpPolicy", env, verbose=1,ent_coef=0.01,tensorboard_log = logdir) #Change training depending on algorithm / architecture / hyperparameters
if not os.path.exists(models_dir):
    os.makedirs(models_dir)

if not os.path.exists(logdir):
    os.makedirs(logdir)

print("========================================")
print("Starting policy learning")
print("========================================")

model.learn(total_timesteps=1000000)

print("========================================")
print("Job Finished.") 
print("========================================")

model.save("CenterReachOut.pickle")

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


c:\Users\matte\anaconda3\envs\myosuite\lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


Starting policy learning
Logging to logs\PPO_364
----------------------------------
| rollout/           |           |
|    ep_len_mean     | 300       |
|    ep_rew_mean     | -1.44e+04 |
| time/              |           |
|    fps             | 902       |
|    iterations      | 1         |
|    time_elapsed    | 2         |
|    total_timesteps | 2048      |
----------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 300          |
|    ep_rew_mean          | -1.44e+04    |
| time/                   |              |
|    fps                  | 638          |
|    iterations           | 2            |
|    time_elapsed         | 6            |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0055579897 |
|    clip_fraction        | 0.0457       |
|    clip_range           | 0.2          |
|    entropy_loss         | -8.53        |

c:\Users\matte\anaconda3\envs\myosuite\lib\site-packages\numpy\core\fromnumeric.py:86: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 300          |
|    ep_rew_mean          | -1.24e+04    |
| time/                   |              |
|    fps                  | 561          |
|    iterations           | 190          |
|    time_elapsed         | 692          |
|    total_timesteps      | 389120       |
| train/                  |              |
|    approx_kl            | 0.0057670367 |
|    clip_fraction        | 0.0299       |
|    clip_range           | 0.2          |
|    entropy_loss         | -9.74        |
|    explained_variance   | 0            |
|    learning_rate        | 0.0003       |
|    loss                 | 1.68e+05     |
|    n_updates            | 1890         |
|    policy_gradient_loss | -0.0043      |
|    std                  | 1.24         |
|    value_loss           | 3.11e+05     |
------------------------------------------
-----------------------------------------
| rollout/  

<h2> Load sb3 model <h2>

In [ ]:
import numpy as np
import myosuite
import gym
from stable_baselines3 import PPO
import time
from IPython.display import clear_output

env = gym.make('CenterReachOut1-v0')
env.set_perturbation_force(0)

model.load('CenterReachOut.pickle')

# evaluate policy
action_arr = {}
all_rewards = []
vel_arr={}
pos_arr={}
hpos_arr={}
lengths=[]
done=[]
for i in range(10000): # test 10000 random targets
  if(i%100==0):
    print(i)
  ep_rewards = []
  done = False
  obs = env.reset()
  action_arr[i] = np.array([0,0,0,0,0,0])
  vel_arr[i]=np.array([0])
  pos_arr[i]=np.array([0])
  hpos_arr[i]=np.array([0,0])
  while not done:
    env.mj_render()
    # get the next action from the policy
    action, _ = model.predict(obs)
    action_arr[i]=np.vstack((action_arr[i], action))
    vel_arr[i]=np.vstack((vel_arr[i], (np.sqrt((env.get_obs_dict(env.sim)['hand_qvel'][0])**2+(env.get_obs_dict(env.sim)['hand_qvel'][1])**2))))
    hpos_arr[i]=np.vstack((hpos_arr[i], [env.get_obs_dict(env.sim)['hand_qpos'][0],env.get_obs_dict(env.sim)['hand_qpos'][1]]))
    pos_arr[i]=np.vstack((pos_arr[i], np.linalg.norm(env.get_obs_dict(env.sim)['reach_err'], axis=-1)))
    
    obs, reward, done, info = env.step(action)
    ep_rewards.append(reward)
    #time.sleep(.05)
  hpos_arr[i] = hpos_arr[i][1:len(hpos_arr[i])]
  vel_arr[i] = vel_arr[i][1:len(vel_arr[i])]
  action_arr[i] = action_arr[i][1:len(action_arr[i])]
  pos_arr[i] = pos_arr[i][1:len(pos_arr[i])]
  all_rewards.append(np.sum(ep_rewards))
  lengths.append(len(action_arr[i]))
print(f"Average reward: {np.mean(all_rewards)} over 10000 episodes")
print(pos_arr)

In [ ]:
import matplotlib.pyplot as plt
n_filter_up = 270
n_filter_down = 8

plt.hist(lengths, bins=300)  # Adj|ust the number of bins as needed
plt.xlabel('Lengths')
plt.ylabel('Frequency')
plt.title('Histogram of Lengths')
plt.show()
count = 0
for length in lengths:
    if length > n_filter_up or length < n_filter_down:
        count += 1
print(count)

percentage = (count / len(lengths)) * 100
print(f"The percentage of values outside the range: {percentage}%")

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
n_ep=10000
avgarr = np.zeros((50,6))
avgvel = np.zeros((50,1))
hposarr = np.zeros((50,2))
for i in range(50):
        a=float(0.00)
        for k in range(n_ep):
            if lengths[k] < n_filter_up and lengths[k] > n_filter_down:
                try:
                    a=a+vel_arr[k][i][0]
                except:
                    pass
            else:
                pass
        avgvel[i] = (a)/(n_ep-count)
avgvel = np.array(avgvel)e
print(avgvel)
avgpos = np.zeros((50,1))
for i in range(50):
        a=float(0.00)
        for k in range(n_ep):
            if lengths[k] < n_filter_up and lengths[k] > n_filter_down:
                try:
                    a=a+pos_arr[k][i][0]
                except:
                    pass
            else:
                pass
        avgpos[i] = (a)/(n_ep-count)
avgpos = np.array(avgpos)
print(avgpos)
for i in range(50):
    for j in range(6):
        a=0
        for k in range(n_ep):
            if lengths[k] < n_filter_up and lengths[k] > n_filter_down:
                try:
                    a=a+action_arr[k][i][j]
                except:
                    pass
            else:
                pass
        avgarr[i][j] = (a)/(n_ep-count)
avgarr = np.array(avgarr)
print(avgarr)

for i in range(50):
    for j in range(2):
        a=0
        for k in range(n_ep):
            if lengths[k] < n_filter_up and lengths[k] > n_filter_down:
                try:
                    a=a+hpos_arr[k][i][j]
                except:
                    pass
            else:
                pass
        hposarr[i][j] = (a)/(n_ep-count)
hposarr = np.array(hposarr)
print(avgarr)

fig, axs = plt.subplots(2, 4, figsize=(16, 8))
fig.subplots_adjust(hspace=0.5, wspace=0.3)

arr=np.array([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50])

col1 = avgarr[:, 0].T
col2 = avgarr[:, 1].T
col3 = avgarr[:, 2].T
col4 = avgarr[:, 3].T
col5 = avgarr[:, 4].T
col6 = avgarr[:, 5].T
col1 = col1.reshape(-1)
col2 = col2.reshape(-1)
col3 = col3.reshape(-1)
col4 = col4.reshape(-1)
col5 = col5.reshape(-1)
col6 = col6.reshape(-1)
avgvel = avgvel.reshape(-1)
avgpos = avgpos.reshape(-1)

df = pd.DataFrame({
    'arr': arr,
    'col1': col1,
    'col2': col2,
    'col3': col3,
    'col4': col4,
    'col5': col5,
    'col6': col6,
    'avgvel': avgvel,
    'avgpos': avgpos
})
df.to_csv('300ms.csv', index=False)

axs[0, 0].plot(arr, avgvel)
axs[0, 0].set_title('Hand Velocity')

axs[0, 1].plot(arr, avgpos)
axs[0, 1].set_title('Distance to Target')

axs[0, 2].plot(arr, col1)
axs[0, 2].set_title('Shoulder Flexor')

axs[0, 3].plot(arr, col2)
axs[0, 3].set_title('Shoulder Extensor')

axs[1, 0].plot(arr, col3)
axs[1, 0].set_title('Elbow Flexor')

axs[1, 1].plot(arr, col4)
axs[1, 1].set_title('Elbow Extensor')

axs[1, 2].plot(arr, col5)
axs[1, 2].set_title('Biarticulate Proximal')

axs[1, 3].plot(arr, col6)
axs[1, 3].set_title('Biarticulate Distal')

plt.show()


In [ ]:
for jj in range(100):
  import matplotlib.pyplot as plt
  import numpy as np
  import myosuite
  import gym
  from stable_baselines3 import PPO
  import time
  from IPython.display import clear_output

  env = gym.make('CenterReachOut1-v0')

  model.load('CenterReachMusclebi_0-1.pickle')

  # evaluate policy
  action_arr = {}
  all_rewards = []
  vel_arr={}
  pos_arr={}
  hpos_arr={}
  lengths=[]
  for i in range(1): # test 10000 random targets
    if(i%100==0):
      print(i)
    ep_rewards = []
    done = False
    obs = env.reset()
    action_arr[i] = np.array([0,0,0,0,0,0])
    vel_arr[i]=np.array([0])
    pos_arr[i]=np.array([0])
    hpos_arr[i]=np.array([0,0])
    while not done:
      env.mj_render()
      # get the next action from the policy
      action, _ = model.predict(obs)
      action_arr[i]=np.vstack((action_arr[i], action))
      vel_arr[i]=np.vstack((vel_arr[i], (np.sqrt((env.get_obs_dict(env.sim)['hand_qvel'][0])**2)+np.sqrt((env.get_obs_dict(env.sim)['hand_qvel'][1])**2))))
      hpos_arr[i]=np.vstack((hpos_arr[i], [env.get_obs_dict(env.sim)['hand_qpos'][0],env.get_obs_dict(env.sim)['hand_qpos'][1]]))
      pos_arr[i]=np.vstack((pos_arr[i], np.linalg.norm(env.get_obs_dict(env.sim)['reach_err'], axis=-1)))
      
      obs, reward, done, info = env.step(action)
      ep_rewards.append(reward)
      time.sleep(.03)
    all_rewards.append(np.sum(ep_rewards))
    lengths.append(len(action_arr[i]))
  print(f"Average reward: {np.mean(all_rewards)} over 10000 episodes")
  hposarr=np.zeros((300,2))
  for i in range(300):
      for j in range(2):
          a=0
          for k in range(1):
              if lengths[k] < 300:
                  try:
                      a=a+hpos_arr[k][i][j]
                  except:
                      pass
              else:
                  pass
          hposarr[i][j] = a
  hposarr = np.array(hposarr)
  fig, axs = plt.subplots(1, 1, figsize=(4, 4))

  arr=np.array([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105])

  col1 = hposarr[:, 0].T
  col2 = hposarr[:, 1].T
  col1 = col1.reshape(-1)
  col2 = col2.reshape(-1)

  plt.scatter(col1, col2)
  file_name = f'sample_plot_{jj}.png'
  plt.savefig(file_name)
  plt.close()

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Create your subplots
fig, axs = plt.subplots(2, 4, figsize=(16, 8))
fig.subplots_adjust(hspace=0.5, wspace=0.3)

df1 = pd.read_csv('300ms.csv')
df2 = pd.read_csv('500ms.csv')
#df3 = pd.read_csv('700k.csv')
#df4 = pd.read_csv('900k.csv')

axs[0, 0].plot(df1['arr'], df1['avgvel'], label='300ms')
axs[0, 1].plot(df1['arr'], df1['avgpos'], label='300ms')
axs[0, 2].plot(df1['arr'], df1['col1'], label='300ms')
axs[0, 3].plot(df1['arr'], df1['col2'], label='300ms')
axs[1, 0].plot(df1['arr'], df1['col3'], label='300ms')
axs[1, 1].plot(df1['arr'], df1['col4'], label='300ms')
axs[1, 2].plot(df1['arr'], df1['col5'], label='300ms')
axs[1, 3].plot(df1['arr'], df1['col6'], label='300ms')

axs[0, 0].plot(df1['arr'], df2['avgvel'], label='500ms')
axs[0, 1].plot(df1['arr'], df2['avgpos'], label='500ms')
axs[0, 2].plot(df1['arr'], df2['col1'], label='500ms')
axs[0, 3].plot(df1['arr'], df2['col2'], label='500ms')
axs[1, 0].plot(df1['arr'], df2['col3'], label='500ms')
axs[1, 1].plot(df1['arr'], df2['col4'], label='500ms')
axs[1, 2].plot(df1['arr'], df2['col5'], label='500ms')
axs[1, 3].plot(df1['arr'], df2['col6'], label='500ms')

'''
axs[0, 0].plot(df1['arr'], df3['avgvel'], label='700k')
axs[0, 1].plot(df1['arr'], df3['avgpos'], label='700k')
axs[0, 2].plot(df1['arr'], df3['col1'], label='700k')
axs[0, 3].plot(df1['arr'], df3['col2'], label='700k')
axs[1, 0].plot(df1['arr'], df3['col3'], label='700k')
axs[1, 1].plot(df1['arr'], df3['col4'], label='700k')
axs[1, 2].plot(df1['arr'], df3['col5'], label='700k')
axs[1, 3].plot(df1['arr'], df3['col6'], label='700k')

axs[0, 0].plot(df1['arr'], df4['avgvel'], label='900k')
axs[0, 1].plot(df1['arr'], df4['avgpos'], label='900k')
axs[0, 2].plot(df1['arr'], df4['col1'], label='900k')
axs[0, 3].plot(df1['arr'], df4['col2'], label='900k')
axs[1, 0].plot(df1['arr'], df4['col3'], label='900k')
axs[1, 1].plot(df1['arr'], df4['col4'], label='900k')
axs[1, 2].plot(df1['arr'], df4['col5'], label='900k')
axs[1, 3].plot(df1['arr'], df4['col6'], label='900k')
'''

# Setting titles
axs[0, 0].set_title('Hand Velocity')
axs[0, 1].set_title('Distance to Target')
axs[0, 2].set_title('Shoulder Flexor')
axs[0, 3].set_title('Shoulder Extensor')
axs[1, 0].set_title('Elbow Flexor')
axs[1, 1].set_title('Elbow Extensor')
axs[1, 2].set_title('Biarticulate Proximal')
axs[1, 3].set_title('Biarticulate Distal')

# Adding legends to each subplot
for ax in axs.flat:
    ax.legend()

# Show the plots
plt.show()

In [ ]:
#Evaluate untrained policy
all_rewards = []
for _ in range(1000): # 20 random targets
  ep_rewards = []
  done = False
  obs = env.reset()
  while not done:
      env.mj_render()
      # take a random action
      obs, reward, done, info = env.step(env.action_space.sample())
      ep_rewards.append(reward)
      #time.sleep(.1)
  all_rewards.append(np.sum(ep_rewards))
print(f"Average reward: {np.mean(all_rewards)} over 1000 episodes")

In [ ]:
pi = PPO.load(policy)
env.reset()
frames = []
env.env.target_type = 'fixed'
o1 = []; o2 = []; o3 = [];4
for _ in range(20):
    env.sim.renderer.render_to_window()
    obs_state = env.get_obs_dict(env.sim)
    o1.append(obs_state["tip_pos"][0]); o2.append(obs_state["tip_pos"][1]); o3.append(obs_state["tip_pos"][2])
    a = pi.predict(o)[0]

    next_o, r, done, ifo = env.step(a) # take an action based on the current observation
    time.sleep(.2)
    time.sleep(.4)

plt.plot(o1)
plt.plot(o2)
plt.plot(o3)
plt.show()

print("Here are the possible values to track:")
simdict = env.get_obs_dict(env.sim)
for key, value in simdict.items():
    print(key, value)